# CAFA 5 protein function Prediction with TensorFlow

The reference  kernel is [here](https://www.kaggle.com/code/gusthema/cafa-5-protein-function-with-tensorflow) on kaggle.


# Labels of the dataset

The objective of our model is to predict the terms (functions) of a protein sequence. One protein sequence can have many functions and can thus be classified into any number of terms. Each term is uniquely identified by a `GO Term ID`. Thus our model has to predict all the `GO Term ID`s for a protein sequence. This means that the task at hand is a multi-label classification problem. 

# Protein embeddings for train and test data

To train a machine learning model we cannot use the alphabetical protein sequences in`train_sequences.fasta` directly. They have to be converted into a vector format. In this notebook, we will use embeddings of the protein sequences to train the model. You can think of protein embeddings to be similar to word embeddings used to train NLP models.
<!-- Instead, to make calculations and data preparation easier we will use precalculated protein embeddings.
 -->
Protein embeddings are a machine-friendly method of capturing the protein's structural and functional characteristics, mainly through its sequence. One approach is to train a custom ML model to learn the protein embeddings of the protein sequences in the dataset being used in this notebook. Since this dataset represents proteins using amino-acid sequences which is a standard approach, we can use any publicly available pre-trained protein embedding models to generate the embeddings.

There are a variety of protein embedding models. To make data preparation easier, we have used the precalculated protein embeddings created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model in this notebook. The precalculated protein embeddings can be found [here](https://www.kaggle.com/datasets/sergeifironov/t5embeds). We have added this dataset to the notebook along with the dataset made available for the competition.

To add this to your enviroment, on the right side panel, click on `Add Data` and search for `t5embeds` (make sure that it's the correct [one](https://www.kaggle.com/datasets/sergeifironov/t5embeds)) and then click on the `+` beside it.



# Import the Required Libraries

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Uses only CPU.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# Changes log level.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf


DIR_INPUT = Path('/kaggle/input')

# Load the Dataset

First we will load the file `train_terms.tsv` which contains the list of annotated terms (functions) for the proteins. We will extract the labels aka `GO term ID` and create a label dataframe for the protein embeddings.

In [ ]:
train_terms = pd.read_csv(
    DIR_INPUT / 'cafa-5-protein-function-prediction/Train/train_terms.tsv',
    sep='\t'
)
print(train_terms.shape)
print(round(train_terms.memory_usage().sum() / 1024**3, 2), 'GB')

`train_terms` dataframe is composed of 3 columns and 5_363_863 entries. We can see all 3 dimensions of our dataset by printing out the first 5 entries using the following code:

In [ ]:
train_terms.head()

# Loading the protein embeddings


We will now load the pre calculated protein embeddings created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model.

If the `tfembeds` is not yet on the input data of the notebook, you can add it to your enviromentby clicking on `Add Data` and search for `t5embeds` (make sure that it's the correct [one](https://www.kaggle.com/datasets/sergeifironov/t5embeds) ) and then click on the `+` beside it.

The protein embeddings to be used for training are recorded in `train_embeds.npy` and the corresponding protein ids are available in `train_ids.npy`.

First, we will load the protein ids of the protein embeddings in the train dataset contained in `train_ids.npy` into a numpy array.

In [ ]:
train_protein_ids = np.load(
    DIR_INPUT / 't5embeds/train_ids.npy'
)
print(train_protein_ids.shape)

The `train_protein_ids` array consists of 142246 protein_ids. Let us print out the first 5 entries using the following code:

In [ ]:
train_protein_ids[:5]

<!-- Now, we will load`train_embeds.py` which contains the pre-calculated embeddings of the proteins in the train dataset. with protein_ids (`id`s we loaded previously from the **train_ids.npy**) into a numpy array. This array now contains the precalculated embeddings for the protein_ids( Ids we loaded above from **train_ids.npy**) needed for training. -->

After loading the files as numpy arrays, we will convert them into Pandas dataframe.

Each protein embedding is a vector of length 1024. We create the resulting dataframe such that there are 1024 columns to represent the values in each of the 1024 places in the vector.

In [ ]:
train_embeddings = np.load(
    DIR_INPUT / 't5embeds/train_embeds.npy'
)

# Now lets convert embeddings numpy array(train_embeddings)
# into pandas dataframe.
column_num = train_embeddings.shape[1]
train_df = pd.DataFrame(
    train_embeddings,
    columns = ["Column_" + str(i) for i in range(1, column_num+1)]
)
# Removes a redundant  object.
del train_embeddings
print(train_df.shape)
print(round(train_df.memory_usage().sum() / 1024**3, 2), 'GB')

The `train_df` dataframe which contains the embeddings is composed of 1024 columns and 142246 entries. We can see all 1024 dimensions(results will be truncated since column length is too long)  of our dataset by printing out the first 5 entries using the following code:

In [ ]:
train_df.head()

# Prepare the dataset

Reference: https://www.kaggle.com/code/alexandervc/baseline-multilabel-to-multitarget-binary

First we will extract all the needed labels(`GO term ID`) from `train_terms.tsv` file. There are more than 40,000 labels. In order to simplify our model, we will choose the most frequent 1500 `GO term ID`s as labels.

Let's plot the most frequent 100 `GO Term ID`s in `train_terms.tsv`.

In [ ]:
# Select first 100 values for plotting.
plot_df = train_terms['term'].value_counts().iloc[:100]

figure, axis = plt.subplots(1, 1, figsize=(12, 6))

bp = sns.barplot(ax=axis, x=np.array(plot_df.index), y=plot_df.values)
bp.set_xticklabels(bp.get_xticklabels(), rotation=90, size = 6)
axis.set_title('Top 100 frequent GO term IDs')
bp.set_xlabel('GO term IDs', fontsize = 12)
bp.set_ylabel('Count', fontsize = 12)
plt.show()

We will now save the first 1500 most frequent GO term Ids into a list.

In [ ]:
# Set the limit for label.
num_of_labels = 1500

# Take value counts in descending order
# and fetch first 1500 `GO term ID` as labels
labels = train_terms['term'].value_counts().index[:num_of_labels].tolist()

Next, we will create a new dataframe by filtering the train terms with the selected `GO Term ID`s.

In [ ]:
# Fetch the train_terms data for the relevant labels only
train_terms_updated = train_terms.loc[train_terms['term'].isin(labels)]
del train_terms

Let us plot the aspect values in the new **train_terms_updated** dataframe using a pie chart.

In [ ]:
pie_df = train_terms_updated['aspect'].value_counts()
palette_color = sns.color_palette('bright')
plt.pie(pie_df.values, labels=np.array(pie_df.index), colors=palette_color, autopct='%.0f%%')
plt.show()

As you can see, majority of the `GO term Id`s have BPO(Biological Process Ontology) as their aspect.

Since this is a multi label classification problem, in the labels array we will denote the presence or absence of each Go Term Id for a protein id using a 1 or 0.
First, we will create a numpy array `train_labels` of required size for the labels. To update the `train_labels` array with the appropriate values, we will loop through the label list.

In [ ]:
# Create an empty dataframe of required size for storing the labels,
# i.e, train_size x num_of_labels (142246 x 1500)
train_size = train_protein_ids.shape[0] # len(X)
train_labels = np.zeros((train_size ,num_of_labels))

# Convert from numpy to pandas series for better handling
series_train_protein_ids = pd.Series(train_protein_ids)

# Loop through each label
for i in range(num_of_labels):
    # For each label, fetch the corresponding train_terms data
    n_train_terms = train_terms_updated[
        train_terms_updated['term'] ==  labels[i]
    ]
    
    # Fetch all the unique EntryId aka proteins related to the current label(GO term ID)
    label_related_proteins = n_train_terms['EntryID'].unique()
    
    # In the series_train_protein_ids pandas series, if a protein is related
    # to the current label, then mark it as 1, else 0.
    # Replace the ith column of train_Y with with that pandas series.
    train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
    
# Convert train_Y numpy into pandas dataframe
labels_df = pd.DataFrame(data = train_labels, columns = labels)
labels_df = labels_df.astype('int32')
print(labels_df.shape)
print(round(labels_df.memory_usage().sum() / 1024**3, 2), 'GB')


The final labels dataframe (`label_df`) is composed of 1500 columns and 142_246 entries. We can see all 1500 dimensions(results will be truncated since the number of columns is big) of our dataset by printing out the first 5 entries using the following code:

In [ ]:
labels_df.head()

# Training

Next, we will use Tensorflow to train a Deep Neural Network with the protein embeddings.

In [ ]:
INPUT_SHAPE = [train_df.shape[1]]
BATCH_SIZE = 5120

model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),    
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=num_of_labels,activation='sigmoid')
])


# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()],
)

history = model.fit(
    train_df, labels_df,
    batch_size=BATCH_SIZE,
    epochs=5
)
# Removes redundant objects.
del train_df
del labels_df

# Submission

For submission we will use the protein embeddings of the test data created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model.

In [ ]:
test_embeddings = np.load(
    DIR_INPUT / 't5embeds/test_embeds.npy'
)

# Convert test_embeddings to dataframe
column_num = test_embeddings.shape[1]
test_df = pd.DataFrame(test_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
print(test_df.shape)
print(test_df.memory_usage().sum() / 1024**3, 'GB')
print(round(test_df.memory_usage().sum() / 1024**3, 2), 'GB')

The `test_df` is composed of 1024 columns and 141865 entries. We can see all 1024 dimensions(results will be truncated since column length is too long) of our dataset by printing out the first 5 entries using the following code:

In [ ]:
test_df.head()

We will now use the model to make predictions on the test embeddings. 

In [ ]:
predictions = model.predict(test_df)
# Removes a redundant  object.
del test_df
print(f'`predictions` shape: {predictions.shape}')
# Rounding of probabilities
# according to the requirements of the competition rules.
predictions = predictions.round(3)
predictions

In [ ]:
# Loads protein IDs from embedded test data.
test_protein_ids = np.load(
    DIR_INPUT / 't5embeds/test_ids.npy'
)
assert test_protein_ids.shape[0] == predictions.shape[0], (
    'Number protein ids in embedded list must be equal to proteins number in predictions.'
)
test_protein_ids

Creates from `predictions` `submission.tsv` file in streaming writing manner.

In [ ]:
submission_chunk_df = pd.DataFrame(
    columns = ['Protein Id', 'GO Term Id','Prediction']
)
for prot_idx, prot_id in enumerate(test_protein_ids, 0):
    prot_chunk = [prot_id] * predictions.shape[1]

    submission_chunk_df['Protein Id'] = [prot_id] * predictions.shape[1]
    submission_chunk_df['GO Term Id'] = labels
    submission_chunk_df['Prediction'] = predictions[prot_idx]

    submission_chunk_df.to_csv(
        '/kaggle/working/submission.tsv',
        header=False,
        index=False,
        mode='a',
        sep='\t'
    )
